# Prereqs

First thing that we're going to do is to set up and install of the prereqs.

##Go over to Files -> New -> Terminal

To copy stuff over you'll use `ssh2.seas.wustl.edu`

From the new terminal, type in `kinit` and then enter in your wustl password.

That should grant access to `/storage1/noguchik/Active/cellquant` which is where the files are stored

If working on RIS servers, then it is `/storage1/fs1/noguchik/Active/cellquant`



TODO: Determine which aren't needed

In [1]:
!pip install --upgrade pip opencv-python pyqt5 lxml wget google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable


These are the global variables that we'll need

In [2]:
import os
import wget
import tarfile
import zipfile

BASE_DIRECTORY=""
try:
  from google.colab import drive
  drive.mount('/content/drive')
  BASE_DIRECTORY=os.path.join(os.getcwd(),'drive','MyDrive','TensorFlow')
except ImportError:
  os.makedirs("content", exist_ok=True)
  BASE_DIRECTORY="/storage1/noguchik/Active/cellquant/TensorFlow"
    
os.environ["LD_LIBRARY_PATH"] = os.environ["LD_LIBRARY_PATH"] + ":/usr/local/cuda/lib64"
UNIQUE_MODIFIER='2021-10-05'
CUSTOM_MODEL_NAME = 'my_centernet' + '_' + UNIQUE_MODIFIER
PRETRAINED_MODEL_NAME = 'centernet_hg104_512x512_kpts_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join(BASE_DIRECTORY, 'workspace'),
    
    'ANNOTATION_PATH': os.path.join(BASE_DIRECTORY, 'workspace','annotations', UNIQUE_MODIFIER),
    
    'MODEL_PATH' : os.path.join(BASE_DIRECTORY, 'workspace','models'),
    'CHECKPOINT_PATH': os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(BASE_DIRECTORY, 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    
    'IMAGE_PATH': os.path.join(BASE_DIRECTORY, 'workspace','images', UNIQUE_MODIFIER),
    'PRETRAINED_MODEL_PATH': os.path.join(BASE_DIRECTORY, 'workspace','pre-trained-models'),
    
    
    'SCRIPTS_PATH': os.path.join(BASE_DIRECTORY,'scripts'),

    'PROTOC_PATH':os.path.join(BASE_DIRECTORY,'protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(paths['WORKSPACE_PATH'],'models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

TRAIN_IMAGES = os.path.join(paths['IMAGE_PATH'], 'train')
TEST_IMAGES = os.path.join(paths['IMAGE_PATH'], 'test')

Make sure all directories exist

In [3]:
for path in paths.values():
  os.makedirs(path, exist_ok=True)

os.makedirs(CUSTOM_MODEL_NAME, exist_ok=True)

Change directories

In [4]:
os.chdir(BASE_DIRECTORY)
print(os.getcwd())

/storage1/noguchik/Active/cellquant/TensorFlow


Check if we need to clone repos. If we do, then do it.

In [5]:
if not os.path.isdir(os.path.join(BASE_DIRECTORY, 'models')):
  try:
    from git import Repo
    Repo.clone_from('https://github.com/tensorflow/models.git', 'models')
  except ImportError:
    !git clone https://github.com/tensorflow/models.git
    
#Update the paths array

paths['APIMODEL_PATH'] = os.path.join(BASE_DIRECTORY,'models')

TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [6]:
!pwd

/storage1/noguchik/Active/cellquant/TensorFlow


# TensorFlow API installation

In [7]:
if os.name=='posix':
    #This was is better since it's archicture independent, but it requires knowing if the OS is Debian based
    #!apt-get install protobuf-compiler
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.18.0/protoc-3.18.0-linux-x86_64.zip"
    file_name = wget.download(url)
    os.rename(file_name,os.path.join(paths['PROTOC_PATH'],file_name))
    os.chdir(paths['PROTOC_PATH'])

    with zipfile.ZipFile(os.path.join(paths['PROTOC_PATH'],file_name), "r") as zip_ref:
        zip_ref.extractall(paths['PROTOC_PATH'])
    !chmod +x {paths['PROTOC_PATH']}/bin/*
    
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))  
    !cd {BASE_DIRECTORY}/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd {BASE_DIRECTORY}/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd {BASE_DIRECTORY}/models/research/slim && pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Processing /storage1/noguchik/Active/cellquant/TensorFlow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1665401 sha256=544b61e8e405b17382aa00962521fad6ce97ee6030e3cebc09fdffe8f8b74def
  Stored in directory: /tmp/pip-ephem-wheel-cache-l77moomj/wheels/88/dc/d9/c682a967dc7d9e25c41d5bde8b7f162bbdee8cdecc0f8bbe89
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-d

# Verify Installation

In [8]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.7.7: /project/compute/anaconda3/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-10-07 16:53:16.164433: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-07 16:53:17.233471: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10421 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b2:00.0, compute capability: 6.1
2021-10-07 16:53:17.234963: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10421 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b1:00.0, comput

Upgrade Tensorflow

In [9]:
os.environ["TMPDIR"]="/home/research/ceneblock"
%pip install --upgrade tensorflow tensorflow-gpu
%pip uninstall protobuf matplotlib -y
%pip install protobuf matplotlib==3.2
%pip install --upgrade protobuf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Found existing installation: protobuf 3.18.1
Uninstalling protobuf-3.18.1:
  Successfully uninstalled protobuf-3.18.1
Found existing installation: matplotlib 3.2.0
Uninstalling matplotlib-3.2.0:
  Successfully uninstalled matplotlib-3.2.0
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached matplotlib-3.2.0-cp37-cp37m-manylinux1_x86_64.whl (12.4 MB)
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-3.18.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


# Training

Grab a model

TODO: check if the downloaded file exists. Downloading a new model takes seconds, so it's not a top priority

In [10]:
import tarfile, wget
downloaded_file = wget.download(PRETRAINED_MODEL_URL, out=paths['PRETRAINED_MODEL_PATH'])
tar = tarfile.open(downloaded_file)
tar.extractall(path=paths['PRETRAINED_MODEL_PATH'])
tar.close()

Create the label map

In [11]:
#labels = [{'name':'Oligodendrocyte', 'id':1}, {'name':'Question', 'id':2}, {'name':'Interneuron', 'id':3}, {'name':'Pyramidal', 'id':4}, {'name':'Axon', 'id':5}]
#
#with open(files['LABELMAP'], 'w') as f:
#    for label in labels:
#        f.write('item { \n')
#        f.write('\tname:\'{}\'\n'.format(label['name']))
#        f.write('\tid:{}\n'.format(label['id']))
#        f.write('}\n')
labels = [{'name':'Neuron', 'id':1}, {'name':'Granule Cell', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

Get the tfrecord script

In [12]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

!python {files['TF_RECORD_SCRIPT']} -i {TRAIN_IMAGES} -x {os.path.join(TRAIN_IMAGES)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -i {TEST_IMAGES} -x {os.path.join(TEST_IMAGES)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

Successfully created the TFRecord file: /storage1/noguchik/Active/cellquant/TensorFlow/workspace/annotations/2021-10-05/train.record
Successfully created the TFRecord file: /storage1/noguchik/Active/cellquant/TensorFlow/workspace/annotations/2021-10-05/test.record


# Start up Tensorboard

In [13]:
#!pip uninstall tensorflow-tensorboard -y
#%load_ext tensorboard.notebook
#%tensorboard --logdir logs

Copy and setup the pipeline config
#TODO this section is broken.

In [ ]:
import shutil
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format



shutil.copy(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config'), os.path.join(paths['CHECKPOINT_PATH']))

config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

config

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]



config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=100000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
!time {command}

2021-10-07 16:57:18.141910: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-07 16:57:19.181111: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10421 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b2:00.0, compute capability: 6.1
2021-10-07 16:57:19.182557: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10421 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b1:00.0, compute capability: 6.1
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/r

In [ ]:
from keras import backend as K 
K.clear_session() 

In [ ]:
print(BASE_DIRECTORY)

# Evaluation

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
!time {command}